# Session 1 – Démarrage du Chat (Foundry Local)

Ce notebook initialise Foundry Local, télécharge l'alias du modèle préféré, et effectue à la fois une complétion de chat standard et une complétion de chat en streaming.


# Scénario
Cette session présente le strict minimum pour faire répondre un petit modèle de langage local via Foundry Local. Vous allez :
- Installer le SDK / les dépendances client.
- Initialiser le gestionnaire Foundry Local pour un alias choisi (par défaut : `phi-4-mini`).
- Appliquer un correctif défensif pour gérer les champs optionnels dans les métadonnées du modèle.
- Envoyer une requête standard de complétion de chat.
- Diffuser une réponse, token par token.

L'objectif est de valider votre environnement local et le chemin réseau avant de passer à RAG, au routage ou aux agents.


### Explication : Installation des dépendances
Installe les packages Python nécessaires pour ce flux de discussion minimal :
- `foundry-local-sdk` : Gérer les modèles locaux et le cycle de vie des services.
- `openai` : Abstraction client familière pour les complétions de chat.
- `rich` : Affichage amélioré pour une sortie plus claire dans les notebooks.

Relancer est sans danger (idempotent). Passez cette étape si votre environnement dispose déjà de ces packages.


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Explication : Importations principales
Introduit les modules utilisés dans tout le notebook :
- `FoundryLocalManager` pour interagir avec l'environnement d'exécution du modèle local.
- Client `OpenAI` afin de réutiliser l'interface API de complétion de chat bien connue.
- `rich.print` pour des sorties stylisées.

Aucun appel réseau ne se produit ici—cela prépare simplement l'espace de noms.


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### Explication : Initialisation du gestionnaire et correctif de métadonnées
Initialise `FoundryLocalManager` pour l'alias choisi et applique un correctif défensif pour gérer avec élégance les réponses du service où `promptTemplate` pourrait être `null`.

Résultats clés :
- Confirme l'état du service et le point de terminaison.
- Liste les modèles mis en cache (vérifie le stockage local).
- Résout l'ID du modèle concret pour l'alias (utilisé dans les appels de chat ultérieurs).

Si vous rencontrez des problèmes de validation dans les métadonnées brutes du service, ce modèle montre comment les assainir sans modifier le SDK.


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### Explication : Complétion de chat basique
Crée un client compatible avec `OpenAI` en se connectant au point de terminaison local de Foundry et effectue une seule complétion de chat non-streaming. Points d'attention :
- S'assurer que le modèle répond sans erreur.
- Valider la latence / le format de sortie.
- Garder `max_tokens` modeste pour économiser les ressources.

Si cela échoue, vérifiez à nouveau que le service Foundry Local est en cours d'exécution et que l'alias se résout correctement.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### Explication : Complétion de chat en streaming
Montre le streaming de tokens pour améliorer la latence perçue et l'expérience utilisateur interactive. La boucle affiche des deltas incrémentaux au fur et à mesure qu'ils arrivent :
- Utile pour les interfaces de chat où une sortie partielle précoce est importante.
- Permet de mesurer le débit des tokens par rapport à la latence de la complétion complète.

Vous pouvez adapter ce modèle pour accumuler des tokens, mettre à jour un widget de progression ou interrompre la génération en cours.


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous ne sommes pas responsables des malentendus ou des interprétations erronées résultant de l'utilisation de cette traduction.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
